# Determining the CDS Basket Spread

An analysis of pricing a CDS Basket 

## NOTE THAT THE API WILL CHANGE BUT THE UNDERLYING MODELS WILL REMAIN

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.credit import *
from financepy.products.rates import *
from financepy.utils import *

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 13:18 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



In [3]:
valuation_date = Date(1, 8, 2007)
settlement_date = valuation_date.add_weekdays(1)

## Build Ibor Curve

In [4]:
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
depo = IborDeposit(settlement_date, "1D", 0.0502, dcType)
depos = [depo]
swap1 = IborSwap(settlement_date,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = IborSwap(settlement_date,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = IborSwap(settlement_date,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = IborSwap(settlement_date,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = IborSwap(settlement_date,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [5]:
step_in_date = valuation_date.add_weekdays(0)
valuation_date = step_in_date

In [6]:
maturity3Y = valuation_date.next_cds_date(36)
maturity5Y = valuation_date.next_cds_date(60)
maturity7Y = valuation_date.next_cds_date(84)
maturity10Y = valuation_date.next_cds_date(120)

### Heterogeneous Curves

In [8]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])
    cds3Y = CDS(step_in_date, maturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, maturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, maturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, maturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(valuation_date, cds_contracts, libor_curve, recovery_rate)
    heteroIssuerCurves.append(issuer_curve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [9]:
homoIssuerCurves = []
num_credits = 125
recovery_rate = 0.40 

In [12]:
cdsIndex = CDSIndexPortfolio()

In [13]:
spd3Y = cdsIndex.intrinsic_spread(valuation_date, step_in_date, maturity3Y, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsic_spread(valuation_date, step_in_date, maturity5Y, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsic_spread(valuation_date, step_in_date, maturity7Y, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsic_spread(valuation_date, step_in_date, maturity10Y, heteroIssuerCurves)

In [14]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.67881818373111
Homogeneous curve 5Y: 35.539099512634195
Homogeneous curve 7Y: 49.01165177405258
Homogeneous curve 10Y: 61.41331504013942


In [15]:
for row in range(0,num_credits):
    cds3Y = CDS(step_in_date, maturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, maturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, maturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, maturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(valuation_date, cds_contracts, libor_curve, recovery_rate)
    homoIssuerCurves.append(issuer_curve)

## Define the Baskets

In [16]:
num_credits = 5

In [17]:
issuer_curves = heteroIssuerCurves[0:num_credits]

In [19]:
basketMaturity = Date(20, 12, 2011)
cdsIndex = CDSIndexPortfolio()

In [21]:
intrinsicSpd = cdsIndex.intrinsic_spread(valuation_date, step_in_date, basketMaturity, issuer_curves) * 10000.0

print("INTRINSIC SPD BASKET MATURITY", intrinsicSpd)

totalSpd = cdsIndex.total_spread(valuation_date, step_in_date, basketMaturity, issuer_curves) * 10000.0

print("SUMMED UP SPD BASKET MATURITY", totalSpd)

minSpd = cdsIndex.min_spread(valuation_date, step_in_date, basketMaturity, issuer_curves) * 10000.0

print("MINIMUM SPD BASKET MATURITY", minSpd)

maxSpd = cdsIndex.max_spread(valuation_date, step_in_date, basketMaturity, issuer_curves) * 10000.0

print("MAXIMUM SPD BASKET MATURITY", maxSpd)

INTRINSIC SPD BASKET MATURITY 29.035439080042693
SUMMED UP SPD BASKET MATURITY 145.79732756821284
MINIMUM SPD BASKET MATURITY 9.61956584754749
MAXIMUM SPD BASKET MATURITY 73.20319094403649


In [22]:
basket = CDSBasket(valuation_date,basketMaturity)

## Gaussian Copula Model

WARNING: THE INTERFACE ON THIS FUNCTION WILL CHANGE SOON! THEY ALSO NEED TO BE ADAPTED TO USE NUMBA.

In [27]:
seed = 42
doF = 5
num_trials = 5000

print("NTrials   Rho    NTD    SPD_GC_MC    SPD_1FGC    SPD_ST10")
print("=========================================================")

for ntd in range(1, num_credits + 1):
    for beta in [0.0, 0.25, 0.5, 0.75, 0.90, 0.9999]:
        rho = beta * beta
        beta_vector = np.ones(num_credits) * beta
        corr_matrix = corr_matrix_generator(rho, num_credits)
        for num_trials in [5000]:
            v1 = basket.value_gaussian_mc(valuation_date,ntd,issuer_curves,corr_matrix,libor_curve,num_trials,seed)
            v2 = basket.value_1f_gaussian_homo(valuation_date,ntd,issuer_curves,beta_vector,libor_curve)
            v3 = basket.value_student_t_mc(valuation_date, ntd, issuer_curves, corr_matrix, doF, libor_curve, num_trials,seed)
            print("%7d  %5.2f    %d    %9.3f   %9.3f    %9.3f"% (num_trials, rho, ntd, v1[2] * 10000, v2[3] * 10000, v3[2] *10000))
    print("=========================================================")


NTrials   Rho    NTD    SPD_GC_MC    SPD_1FGC    SPD_ST10
   5000   0.00    1      136.795     144.115      124.474
   5000   0.06    1      132.078     142.097      121.591
   5000   0.25    1      122.355     133.632      115.075
   5000   0.56    1      105.720     111.965       97.041
   5000   0.81    1       83.279      88.627       81.204
   5000   1.00    1       70.959      73.204       68.675
   5000   0.00    2        4.194       5.056       14.680
   5000   0.06    2        6.282       6.823       16.536
   5000   0.25    2       13.687      13.439       21.588
   5000   0.56    2       24.977      25.498       27.768
   5000   0.81    2       30.660      32.097       29.979
   5000   1.00    2       20.595      22.005       20.719
   5000   0.00    3        0.122       0.086        1.558
   5000   0.06    3        0.377       0.219        2.193
   5000   0.25    3        1.266       1.451        4.469
   5000   0.56    3        7.599       7.506       10.608
   5000   0.81

Copyright (c) 2020 Dominic O'Kane